In [1]:
from pathlib import Path
import random
from tqdm import tqdm
import pandas as pd
import numpy as np
import shutil
from utils.functions import get_video_length
from utils.real_noise import real_noise_config, real_noise

ROOT_DIR = Path("/home/sharing/disk1/Datasets/MSA/MMSA_Standard/MSA_RT/Raw/")
SEED = 40

video_paths = list((ROOT_DIR / "original").glob("*.mp4"))
video_paths.sort()

def setup_seed(seed):
    random.seed(seed)
    np.random.seed(seed)

In [2]:
data = pd.read_csv("/home/sharing/disk1/Datasets/MSA/MMSA_Standard/MSA_RT/label.csv")
data['annotation'].value_counts()

FileNotFoundError: [Errno 2] No such file or directory: '/home/sharing/disk1/Datasets/MSA/MMSA_Standard/MSA_RT/label.csv'

# Missing

### 视频缺失

In [3]:
setup_seed(SEED)
missing_rates = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]
for missing_rate in missing_rates:
    (Path(ROOT_DIR) / "video_missing" / str(missing_rate)).mkdir(parents=True, exist_ok=True)
    pbar = tqdm(video_paths)
    pbar.set_description(f"Missing rate: {missing_rate}")
    for v_path in pbar:
        cfg = real_noise_config(
            v_path, 
            mode = "random_full", 
            v_noise_list=["blank"], 
            v_noise_num = 1 if missing_rate in [0.1, 0.2, 0.4, 1.0] else (1, 3),
            v_noise_ratio = missing_rate,
            a_noise_list = [],
        )
        out_path = Path(ROOT_DIR) / "video_missing" / str(missing_rate) / v_path.name
        out_path.parent.mkdir(parents=True, exist_ok=True)
        real_noise(v_path, out_path, **cfg._asdict())

Missing rate: 1.0: 100%|██████████| 62/62 [01:32<00:00,  1.49s/it]


### 音频缺失

In [4]:
setup_seed(SEED)
missing_rates = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]
for missing_rate in missing_rates:
    (Path(ROOT_DIR) / "audio_missing" / str(missing_rate)).mkdir(parents=True, exist_ok=True)
    pbar = tqdm(video_paths)
    pbar.set_description(f"Missing rate: {missing_rate}")
    for v_path in pbar:
        cfg = real_noise_config(
            v_path, 
            mode = "random_full", 
            v_noise_list=[], 
            a_noise_list = ["mute"],
            a_noise_num = 1 if missing_rate in [0.1, 0.2, 0.4, 1.0] else (1, 3),
            a_noise_ratio = missing_rate,
        )
        out_path = Path(ROOT_DIR) / "audio_missing" / str(missing_rate) / v_path.name
        out_path.parent.mkdir(parents=True, exist_ok=True)
        real_noise(v_path, out_path, **cfg._asdict())

Missing rate: 1.0: 100%|██████████| 62/62 [00:08<00:00,  7.05it/s]


### 音视频同步缺失

In [5]:
setup_seed(SEED)
missing_rates = [0.1, 0.2, 0.4, 0.6, 0.8]
for missing_rate in missing_rates:
    (Path(ROOT_DIR) / "sync_missing" / str(missing_rate)).mkdir(parents=True, exist_ok=True)
    pbar = tqdm(video_paths)
    pbar.set_description(f"Missing rate: {missing_rate}")
    for v_path in pbar:
        cfg = real_noise_config(
            v_path, 
            mode = "random_full", 
            v_noise_list=["blank"],
            v_noise_num = 1 if missing_rate in [0.1, 0.2, 0.4] else (1, 3),
            v_noise_ratio = missing_rate, 
            a_noise_list = [],
        )._asdict()
        cfg['a_mode'] = ["volume"] * len(cfg['v_mode'])
        cfg['a_option'] = [0.0] * len(cfg['a_mode'])
        cfg['a_start'] = cfg['v_start']
        cfg['a_end'] = cfg['v_end']
        out_path = Path(ROOT_DIR) / "sync_missing" / str(missing_rate) / v_path.name
        out_path.parent.mkdir(parents=True, exist_ok=True)
        real_noise(v_path, out_path, **cfg)

Missing rate: 0.8: 100%|██████████| 62/62 [01:45<00:00,  1.70s/it]


### 音视频异步缺失

In [6]:
setup_seed(SEED)
missing_rates = [0.1, 0.2, 0.4, 0.6, 0.8]
for missing_rate in missing_rates:
    (Path(ROOT_DIR) / "async_missing" / str(missing_rate)).mkdir(parents=True, exist_ok=True)
    pbar = tqdm(video_paths)
    pbar.set_description(f"Missing rate: {missing_rate}")
    for v_path in pbar:
        cfg = real_noise_config(
            v_path, 
            mode = "random_full", 
            v_noise_list=["blank"],
            v_noise_num = 1 if missing_rate in [0.1, 0.2, 0.4] else (1, 3),
            v_noise_ratio = missing_rate, 
            a_noise_list = ["mute"],
            a_noise_num = 1 if missing_rate in [0.1, 0.2, 0.4] else (1, 3),
            a_noise_ratio = missing_rate,
        )
        out_path = Path(ROOT_DIR) / "async_missing" / str(missing_rate) / v_path.name
        out_path.parent.mkdir(parents=True, exist_ok=True)
        real_noise(v_path, out_path, **cfg._asdict())

Missing rate: 0.8: 100%|██████████| 62/62 [01:45<00:00,  1.71s/it]


# Noise

### 视频噪声

In [7]:
setup_seed(SEED)
noise_types = ["blur", "occlusion", "color", "rgb2bgr"]
for noise_type in noise_types:
    out_dir = Path(ROOT_DIR) / "video_noise" / str(noise_type)
    out_dir.mkdir(parents=True, exist_ok=True)
    pbar = tqdm(video_paths)
    pbar.set_description(f"Noise Type: {noise_type}")
    for v_path in pbar:
        cfg = real_noise_config(
            v_path, 
            mode = "random_time", 
            v_noise_list=[noise_type], 
            v_noise_ratio = 1.0,
            v_noise_intensity = (0.4, 0.6),
            a_noise_list = [],
        )
        out_path = out_dir / v_path.name
        out_path.parent.mkdir(parents=True, exist_ok=True)
        real_noise(v_path, out_path, **cfg._asdict())

Noise Type: rgb2bgr: 100%|██████████| 62/62 [02:22<00:00,  2.30s/it]


### 音频噪声

In [9]:
setup_seed(SEED)
# noise_types = ["lowpass", "background", "reverb"]
noise_types = ["background"]
for noise_type in noise_types:
    out_dir = Path(ROOT_DIR) / "audio_noise" / str(noise_type)
    out_dir.mkdir(parents=True, exist_ok=True)
    pbar = tqdm(video_paths)
    pbar.set_description(f"Noise Type: {noise_type}")
    for v_path in pbar:
        cfg = real_noise_config(
            v_path, 
            mode = "random_time", 
            a_noise_list=[noise_type], 
            a_noise_ratio = 1.0,
            a_noise_intensity = (0.5, 0.8) if noise_type != "background" else (0.2, 0.5),
            v_noise_list = [],
        )
        out_path = out_dir / v_path.name
        out_path.parent.mkdir(parents=True, exist_ok=True)
        real_noise(v_path, out_path, **cfg._asdict())

Noise Type: background: 100%|██████████| 62/62 [00:19<00:00,  3.17it/s]
